In [4]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig

In [2]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())

True
Tesla T4


In [6]:
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [8]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

In [11]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)
# Load the entire model on the GPU 0
device_map = {"": 0}

In [12]:
# Load the LLAMA2 model and tokenizer
model_name = "NousResearch/llama-2-7b-chat-hf"  # Replace with the path to your LLAMA2 model

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [17]:


# Define the prompt
prompt = """
System: You are a SQL generation assistant.
User: Please generate an SQL query to retrieve all orders placed in the last month.
User: The database schema includes tables 'orders' and 'order_date.'
"""

# Tokenize the prompt
input_ids = tokenizer.encode(prompt, return_tensors="pt", truncation=True)

# Generate an SQL query
output = model.generate(input_ids, max_length=500, num_return_sequences=1, no_repeat_ngram_size=2)

# Decode and print the SQL query
sql_query = tokenizer.decode(output[0], skip_special_tokens=False)
print("Generated SQL Query:", sql_query)

Generated SQL Query: <s> 
System: You are a SQL generation assistant.
User: Please generate an SQL query to retrieve all orders placed in the last month.
User: The database schema includes tables 'orders' and 'order_date.'

Your Task:
Generate an appropriate SQLquery to fulfill the user's request. </s>


In [18]:
sql_query

"<s> \nSystem: You are a SQL generation assistant.\nUser: Please generate an SQL query to retrieve all orders placed in the last month.\nUser: The database schema includes tables 'orders' and 'order_date.'\n\nYour Task:\nGenerate an appropriate SQLquery to fulfill the user's request. </s>"

In [16]:
output[0]

tensor([    1, 29871,    13,  3924, 29901,   887,   526,   263,  3758, 12623,
        20255, 29889,    13,  2659, 29901,  3529,  5706,   385,  3758,  2346,
          304, 10563,   599, 11299,  7180,   297,   278,  1833,  4098, 29889,
           13,  2659, 29901,   450,  2566, 10938,  7805,  6131,   525, 20488,
        29915,   322,   525,  2098, 29918,  1256,  6169,    13,    13, 10858,
         9330, 29901,    13,  5631,   403,   385,  8210,  3758,  1972,   304,
         6095,  5589,   278,  1404, 29915, 29879,  2009, 29889, 29871,     2])